# Esercitazione 2 - content2form

Studenti:

- Brunello Matteo (mat. 858867)
- Caresio Lorenzo (mat. 836021)

*Consegna*: i comuni dizionari a cui siamo abituati partono dalle parole, ovvero dalla forma, per arrivare al contenuto. Esistono alcuni tipi di dizionario chiamati dizionari analogici che funzionano ”al contrario”, ovvero non si ricerca per parola ma per definizione. Questo tipo di ricerca viene chiamata ricerca onomasiologica, ovvero si parte dal contenuto per arrivare alla forma. Proprio su questo si basa la seconda esercitazione. Sempre partendo dai dati sulle definizioni, si richiede di provare a costruire un sistema che utilizzi la molteplicità delle definizioni per risalire al termine "target" in maniera automatica. Non si richiede di "indovinare" ogni termine, ma di avvicinarsi (almeno semanticamente) alla risposta. Provare più soluzioni, includendo meccanismi di filtro delle definizioni (ad es. escludendo quelle meno informative o con caratteristiche particolari), di ricerca nell'albero tassonomico di WordNet (provando a partire da candidati "genus", secondo il principio Genus-Differentia), ecc.

La soluzione fornita è di seguito formulata nelle sue generalità:

- Per ogni *forma target*, si identifica un *genus* candidato (o un insieme di *genera candidati*) e si raccoglie del materiale lessicale composto dalle parole (lemmatizzate) presenti nella varie descrizioni;
- Per ogni *genus candidato*, si ottiene il (o i) synset di riferimento e ne si raccolgono gli iponimi. Per ogni synset iponimo si raccoglie del materiale lessicale (lemmatizzato) associato: una *firma* composta dai suoi lemmi, dalla glossa e dagli eventuali esempi d'uso;
- Si reitira la raccolta degli iponimi e del loro materiale lessicale per un numero (parametrizzabile) di livelli dell'albero di WordNet (si tratta di una visita in ampiezza della multi-gerarchia WN);
- Infine, per ogni *forma target*, si calcola l'overlap tra il materiale lessicale ottenuto dalle definizioni d'esempio del dataset e tra quello degli iponimi, andando a visualizzare gli iponimi con maggiore overlap (che costituiscono quindi delle *forme candidate*).

In [ ]:
# Open the inspect element into your moodle session, then paste the "MoodleSession" field value in the storage/cookies tab
moodle_session_cookie = 'ifbrmeeravqdsr5itc75gad5kc'

!curl --cookie 'MoodleSession={moodle_session_cookie}' "https://informatica.i-learn.unito.it/pluginfile.php/366022/mod_folder/content/0/TLN-definitions-23.tsv?forcedownload=1" -o definitions.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8514  100  8514    0     0  11618      0 --:--:-- --:--:-- --:--:-- 11615


In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
import string


nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Pre-processing del dataset

Sul dataset dato si effettuano due tipologie di *pre-processing* in base all'utilizzo che se ne vuole fare: per il calcolo finale dell'overlap, è semplicemente necessario tokenizzare le definizioni, lemmatizzarne le parole e filtrare eventuali parole spurie; per il calcolo dei *genera*, oltre alle fasi appena elencate, è necessario andare a valutare solo i sostantivi, e a tale scopo si utilizzerà un PoS Tagger fornito da NLTK.

Attualmente si evita di fare un vero e proprio controllo qualità sulle definizioni del dataset (come proposto come possibilità a lezione), piuttosto, dopo la fase di tokenizzazione, si eliminano invece delle stringhe spurie o delle parole troppo generiche per essere utili (`eg` *object*, o le parole target stesse che si stanno cercando, presenti in alcune definizioni).

In [ ]:
stopwords = set(stopwords.words('english') + list(string.punctuation))
lemmatizer = WordNetLemmatizer()

def tokenize_and_lemmatize_sentence(sentence):
  return [lemmatizer.lemmatize(word) for word in word_tokenize(sentence.lower()) if word not in stopwords and not check_if_bad_word(word)]

# Tokenize a sentence and lemmatize all its noun, filtering out all the stopwords, punctuation marks and the identified bad words
def get_lemmatized_nouns_in_sentence(sentence):
  nouns = [word for (word, pos) in nltk.pos_tag(word_tokenize(sentence.lower())) if(pos[:2] == 'NN')] # Get tokenized nouns via NLTK
  return [lemmatizer.lemmatize(noun) for noun in nouns if noun not in stopwords and not check_if_bad_word(noun)] # Lemmatize nouns and filter bad words

def check_if_bad_word(word):
  return word in ["'s", "n't", 'u', 'ca', 'object', 'entity', 'something', 'door', 'pain'] # Derived empirically by looking at the tokenized dataset

Si carica quindi il dataset e si stampa una selezione di definizioni pre-processate (nelle due modalità) come sopra.

In [ ]:
import csv

# Load definitions excluding the line number,
# returning the result as a dictionary
def load_definitions(path: str, sentence_processor = lambda x: x):
  result = {}
  with open(path, 'r') as f:
    content = list(csv.reader(f, delimiter = '\t'))
    for i, word in enumerate(content[0][1:]):
      result[word] = [sentence_processor(definition[i+1]) for definition in content[1:]]
    return result

In [ ]:
path = 'definitions.tsv'

print("Lemmatized definitions:")
lemmatized_definitions = load_definitions(path, tokenize_and_lemmatize_sentence)
for form, definitions in lemmatized_definitions.items():
  print(" ", form)
  for processed_definition in definitions[:2]:
    print("  ", processed_definition)

print("Lemmatized nouns-only definitions:")
nouns_only_definitions = load_definitions(path, get_lemmatized_nouns_in_sentence)
for form, definitions in nouns_only_definitions.items():
  print(" ", form)
  for processed_definition in definitions[:2]:
    print("  ", processed_definition)

Lemmatized definitions:
  door
   ['construction', 'used', 'divide', 'two', 'room', 'temporarily', 'closing', 'passage']
   ['opening', 'opened', 'closed']
  ladybug
   ['small', 'flying', 'insect', 'typically', 'red', 'black', 'spot', 'six', 'leg']
   ['insect', 'wing', 'red', 'black', 'dot', 'many', 'culture', 'symbol', 'good', 'luck']
  pain
   ['feeling', 'physical', 'mental', 'distress']
   ['feeling', 'physical', 'emotional', 'bad', 'hurt']
  blurriness
   ['sight', 'focus']
   ['absence', 'definite', 'border', 'shapelessness']
Lemmatized nouns-only definitions:
  door
   ['construction', 'room', 'passage']
   ['opening']
  ladybug
   ['insect', 'spot', 'leg']
   ['insect', 'wing', 'dot', 'culture', 'symbol', 'luck']
  pain
   ['feeling', 'distress']
   ['feeling']
  blurriness
   ['focus']
   ['absence', 'border', 'shapelessness']


### Calcolo dei genera

Per lo svolgimento di questa esercitazione si è deciso di seguire il principio *Genus-Differentia*: ogni definizione di un concetto è composta da una parte relativa al *genus*, un concetto più generale a quello che si sta cercando di descrivere, e da una parte relativa alla *differentia*, dove si fornisce la caratterizzazione che distingue il concetto attuale rispetto ad altri a lui simili.

Prendendo l'esempio del dataset, relativo al concetto di *coccinella*, «a red insect with black spots», si può identificare *insect* come il *genus* e *red* e *with black spots* come sua *differentia*.

Per l'identificazione di un *genus* a partire da un insieme di definizioni si è deciso di calcolarne il sostantivo più frequente, che andrà a costituire il *genus candidato* per la data forma (da questo si calcoleranno gli iponimi per il calcolo dell'overlap finale). Per una soluzione più flessibile, al posto di derivare un singolo *genus* da un insieme di definizioni, se ne calcola un numero parametrizzabile (sempre calcolato come l'insieme dei sostantivi più frequenti).

In [ ]:
from collections import Counter

# Compute candidate genera as the nouns with higher frequency in the given definition collection
def compute_candidate_genera(definitions, n=1):
  word_list = []

  # Flatten the definitions words in a single list
  for definition in definitions:
    word_list.extend(definition)

  # Get the most frequent nouns (candidate genera) with their frequencies
  most_frequents_nouns = Counter(word_list).most_common(n)

  return [genus[0] for genus in most_frequents_nouns]

In [ ]:
n_genera = 3 # PARAMETER

candidate_genera_by_form = dict()

for form, definitions in nouns_only_definitions.items():
  candidate_genera_by_form[form] = compute_candidate_genera(definitions, n_genera)
  print(f"Candidate genera for {form}: {candidate_genera_by_form[form]}")

Candidate genera for door: ['room', 'access', 'space']
Candidate genera for ladybug: ['insect', 'dot', 'luck']
Candidate genera for pain: ['feeling', 'sensation', 'discomfort']
Candidate genera for blurriness: ['image', 'eye', 'vision']


È da subito evidente come per i concetti concreti (sia specifici che generali) i *genera candidati* ottenuti possono essere utili a derivare il senso relativo alle *forme target*, così come per *pain*, concetto astratto e generale, mentre per il concetto di *blurriness*, astratto e specifico, la possibilità di derivare la *forma target* dai *genera* ottenuti sarà molto bassa.

### Derivazione delle forme dai genera

Una volta ottenuti i vari *genera candidati* per ogni *forma target*, si va a interrogare WordNet per ottenerne i synset associati. Gli iponimi di quest'ultimi costituiranno delle *forme candidate*, su cui verranno raccolti dei materiali lessicali associati (*firme* composte da lemmi, glosse ed esempi d'uso) e su cui si calcoleranno infine gli overlap.

Non ci si limiterà però solo a valutare gli iponimi del dato *genus candidato*, ma si andrà a valutare anche gli iponimi degli iponimi, e gli iponimi di questi, per un numero di iterazioni parametrizzabile, andando così a visitare (in ampiezza) la multi-gerarchia di WordNet.






In [ ]:
def collect_hyponyms(sense, depth=2):

  max_depth = depth
  stack = [sense]

  collected_hyponyms = []

  # Breadth-First Search over hyponyms
  while depth > 0:
      new_stack = []
      for node in stack:
          hyponyms = node.hyponyms()
          if hyponyms:
            collected_hyponyms.extend(hyponyms)
            new_stack.extend(node.hyponyms())

      stack = new_stack # On the next iteration, the new hyponyms will be evaluated (expanded)
      depth -= 1

  return collected_hyponyms

def collect_lexical_material(sense):
  lexical_material = sense.definition() # Add the definition of the current sense
  for lemma in sense.lemma_names(): # Add all the related lemmas of the current sense
    lexical_material.join(lemma)
  for example in sense.examples(): # Add (if present) examples of the current sense
    lexical_material.join(example)
  return tokenize_and_lemmatize_sentence(lexical_material) # Lemmatize the lexical material in order to compare it later

In [ ]:
def compute_overlap(dataset_lexical_material, hyponym_lexical_material):
  dataset_words = set(dataset_lexical_material)
  hyponym_words = set(hyponym_lexical_material)
  return len(dataset_words.intersection(hyponym_words))

In [ ]:
deepening_depth = 2 # PARAMETER

dataset_lexical_material = {key:[] for key in lemmatized_definitions.keys()}

# Collect the lexical material from the definitions contained in the dataset
for form, definitions in lemmatized_definitions.items():
  for definition in definitions:
    dataset_lexical_material[form].extend(definition)

hyponyms_with_overlap = {key:[] for key in lemmatized_definitions.keys()}

# Compute the hyponyms and their overlap with original definitions
for form, genera in candidate_genera_by_form.items():
  for genus in genera:
    genus_synsets = wordnet.synsets(genus)
    for synset in genus_synsets:
      hyponyms = collect_hyponyms(synset, deepening_depth)
      for hyponym in hyponyms:
        overlap_size = compute_overlap(dataset_lexical_material[form], collect_lexical_material(hyponym))
        if overlap_size > 1:
          hyponyms_with_overlap[form].append((hyponym, overlap_size))

Una volta individuati gli iponimi con un overlap rispetto al materiale lessicale del dataset originale, li si ordina per dimensione dell'overlap e se ne stampa un numero parametrizzabile.

In [ ]:
def get_sorted_hyponyms(hyponyms, n):
  hyponyms.sort(key=lambda a: a[1], reverse=True)
  return hyponyms[:n]

In [ ]:
n_hyponyms = 5 # PARAMETER

for form, hyponyms in hyponyms_with_overlap.items():
  sorted_hyponyms = get_sorted_hyponyms(hyponyms, n_hyponyms)
  print(form)
  for hyponym in sorted_hyponyms:
    print("  ", hyponym)

door
   (Synset('doorway.n.01'), 8)
   (Synset('compartment.n.02'), 4)
   (Synset('booth.n.02'), 4)
   (Synset('box.n.09'), 4)
   (Synset('angle.n.01'), 4)
ladybug
   (Synset('ladybug.n.01'), 5)
   (Synset('dipterous_insect.n.01'), 4)
   (Synset('lacewing.n.01'), 4)
   (Synset('thrips.n.01'), 4)
   (Synset('leaf_miner.n.01'), 3)
pain
   (Synset('pain.n.02'), 3)
   (Synset('suffering.n.04'), 3)
   (Synset('sorrow.n.01'), 3)
   (Synset('tickle.n.01'), 3)
   (Synset('glow.v.05'), 3)
blurriness
   (Synset('acuity.n.01'), 5)
   (Synset('memory_picture.n.01'), 4)
   (Synset('naked_eye.n.01'), 4)
   (Synset('visual_image.n.01'), 3)
   (Synset('collage.n.01'), 3)


Come previsto, per *door*, *ladybug* e *pain* la *forma candidata* con maggiore overlap corrisponde a quella *target*, mentre per *blurriness*, concetto astrattato e specifico, i cui *genera candidati* non risultavano particolarmente azzeccati, le *forme candidate* ottenute risultano associate allo stesso campo semantico della *forma target*, quello della percezione visiva, ma non corrispondono con esattezza al *target* stesso (il synset WordNet associato a *blurriness* è `indistinctness.n.01`).

In generale si può concludere che, vista anche la non particolare sofisticatezza della soluzione adottata, utilizzare il principio *Genus-Differentia* può portare a dei risultati soddisfacenti per il task di *content2form*.